In [20]:
%load_ext autoreload
%autoreload 2

import sys, os
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy import interpolate
from scipy import integrate
from scipy.optimize import differential_evolution
from scipy.special import xlogy
from scipy.optimize import minimize

from astropy import units as u

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Independent Check on Projection for Future SN

Here we provide an independent check on the projections for axion constraints considering a Fermi-LAT-like telescope observing a 10kpc BSG in the future. We assume a KSVZ-like axion (see 2405.19393).

Check performed by Orion Ning.

In [21]:
#read in spectra
#we assume a KSVZ-like axion (see 2405.19393), normalized to gagg = 1e-12 GeV^-1

spec_data = np.load('projection_check.npz')
print(spec_data.files)

#spectra in units of photons/GeV/s

['Primakoff', 'Brem', 'Pion', 'E_bins', 'ma', 'time_bins']


In [22]:
#read in the Fermi effective area

with fits.open("aeff_P8R3_TRANSIENT020_V2_PSF.fits") as f:
#     print(f.info())
    #print(f[0].header)
    
    effA = 0
    for i in range(4):
        effA += f[i * 3 + 1].data["EFFAREA"]
    ebins = np.vstack((f[1].data["ENERG_LO"], f[1].data["ENERG_HI"]))
    ctheta = np.vstack((f[1].data["CTHETA_LO"], f[1].data["CTHETA_HI"]))

ecenters = np.mean(ebins, axis=0) / 1e3
whs_E = np.where(ecenters < 1.0)[0]

f_effA = interpolate.interp1d(ecenters[whs_E], np.amax(effA[0], axis=0)[whs_E] * 10000, fill_value='extrapolate')

In [23]:
class analysis_ON:
    def __init__(self, ma_idx, dist, spec_dict, f_effA, prim_flag = True, brem_flag = True, pion_flag = True, debug_flag=False):
        
        self.spec_dict = spec_dict

        self.debug_flag = debug_flag
        
        self.ma_idx = ma_idx
        
        self.prim_flag = prim_flag
        self.brem_flag = brem_flag
        self.pion_flag = pion_flag
        
        self.d = dist

        self._load_spec(self.ma_idx, self.spec_dict)
        
        if self.prim_flag:
            print('including primakoff')
            
        if self.brem_flag:
            print('including nucleon')
            
        if self.pion_flag:
            print('including pion')

        self.dE = np.concatenate(((0.1 - 0.001)/20 * np.ones(20), (1 - 0.12)/30 * np.ones(30)))
        
        self.f_effA = f_effA #interpolation function for effArea
        self.effA_Fermi_raw = self.f_effA(self.E_bins)
        self.effA_Fermi = np.clip(self.effA_Fermi_raw, 0, None)

        self.data_cts = np.zeros_like(self.E_bins) #for projections, assume no background


        #timing
        self.delta_T_arr = np.diff(self.time_bins)                  
        self.delta_T_arr = np.append(self.delta_T_arr, self.delta_T_arr[-1]) 
        
        self.modnorm = 1.0e-12

        if debug_flag:
            print("self.effA_Fermi: ", self.effA_Fermi)
            print("self.data_cts: ", self.data_cts)
            print("Time bins: " ,self.time_bins)
            print("Time bin widths: ", self.delta_T_arr)
        
        gthresh = 2.71
        self.modnorm = 1.0e-14 #choose normalization
        lim_test = self.getlim(gthresh) 
        lim_lim  = (lim_test * self.modnorm**4.0)**(1.0/4.0)

        print("Limit: ", lim_lim)
        
        
    #####################################LOAD THINGS
    
    def _load_spec(self, ma_idx, _dict): #all normalized to gagg=1e-12 by default
        self.time_bins = _dict['time_bins']
        self.E_bins = _dict['E_bins']
        self.prim = _dict['Primakoff'][ma_idx]
        self.brem = _dict['Brem'][ma_idx]
        self.pion = _dict['Pion'][ma_idx]
        
    ###############################MAKE SIGNAL
    #we'll make a sig_model of energy for each time step
    def make_signal(self, signorm):
        gagg = self.modnorm
        spectra = np.zeros_like(self.E_bins)

        if self.prim_flag:
            prim_photon = signorm * (gagg / 1e-12)**4 * self.prim[self.t_ind] * self.delta_T 
            spectra += prim_photon
            
        if self.brem_flag:
            brem_photon = signorm * (gagg / 1e-12)**4 * self.brem[self.t_ind] *  self.delta_T
            spectra += brem_photon
            
        if self.pion_flag:
            pion_photon = signorm * (gagg / 1e-12)**4 * self.pion[self.t_ind] *  self.delta_T
            spectra += pion_photon
            
        model_cts = self.effA_Fermi * spectra * self.dE / (4*np.pi*self.d**2)
        
        if self.debug_flag:
            print("model cts: ", model_cts)
        
        return model_cts

    ###############################FUNCTIONS FOR MINIMIZATION
    
    def loss(self, signorm):
    
        self.sig_model = self.make_signal(signorm)
    
        if np.min(self.sig_model) < 0.0:
            return 1.0e10
        else:
            out = -2*np.sum(xlogy(self.data_cts, self.sig_model) - self.sig_model)
            return np.nan_to_num(out, nan = 1.0e10)
    
    def joint_loss(self, signorm):
        
        joint_tot = 0

        for ti in range(len(self.time_bins)):
            self.t_ind = ti
            self.delta_T = self.delta_T_arr[ti]
            
            the_loss = self.loss(signorm)
            joint_tot += the_loss
                
        return joint_tot
    
    
    def getlim(self, thresh):
        #determine best-fit value
        findBF = minimize(lambda siglam: self.joint_loss(*siglam), [0], method = 'Nelder-Mead', \
                         options = {'initial_simplex':[[-10.0], [10.0]], 'fatol':1e-4}) #[[-100
        
        bestfit_sig = findBF.x[0]
        bestfit_TS = findBF.fun

        if self.debug_flag:
            print("Found Best Fit: ")
            print("BF normalization: ", bestfit_sig)
            print("BF g_agg: ", (bestfit_sig * self.modnorm**4.0)**(1.0/4.0))
            print("BF TS: ", bestfit_TS)
        
        self.bestfit_sig = bestfit_sig
        self.bestfit_TS = bestfit_TS

        #now we get the limit
        findlim = minimize(lambda siglam: np.abs((self.joint_loss(*siglam) - bestfit_TS)*(siglam[0] > bestfit_sig) - \
                                                 thresh), [bestfit_sig+5.0], method = 'Nelder-Mead', \
                          options = {'initial_simplex':[[bestfit_sig], [bestfit_sig+10.0]], 'fatol':1e-3})
        
        return findlim.x[0]


In [25]:
ma_idx = 0 
fermi_proj = analysis_ON(ma_idx, (10.0 * u.kpc).to(u.cm).value, spec_data, f_effA, \
                         prim_flag = True, brem_flag = True, pion_flag = True, debug_flag=False)

including primakoff
including nucleon
including pion
Limit:  1.6729883858229222e-14


We see that this resulting 95% upper limit reproduces the low-mass KSVZ dotted red line projection in Fig. 1 of 2405.19393. The exact value in that plot is $\sim 1.69\times 10^{-14}$ GeV$^{-1}$ at low-mass. 